In [ ]:
import numpy as np
from scipy.stats import norm, ncx2

### 10.1.1 Black-Scholes Model

In this coding example we implement the Black-Scholes formula in Python for a European call option:

In [ ]:
def bs_call(S_0, K, T, sigma, r):
    d_1 = (np.log(S_0/K) + T*(r + (sigma**2)/2))/(sigma*np.sqrt(T))
    d_2 = d_1 - sigma*np.sqrt(T)
    return S_0*norm.cdf(d_1) - K*np.exp(-r*T)*norm.cdf(d_2)

print(bs_call(S_0 = 100, K = 105, T = 0.5, sigma = 0.3, r = 0.02))

6.779490734346545


### 10.1.4 Bachelier Model

In this coding example we show how to price call options under the Bachelier model in Python:

In [ ]:
def bachelier_call(S_0, K, T, sigma, r):
    d_plus = (S_0*np.exp(r*T) - K)/(sigma*np.sqrt(T))
    return np.exp(-r*T)*sigma*np.sqrt(T)*(d_plus*norm.cdf(d_plus) + norm.pdf(d_plus))

print(bachelier_call(S_0 = 100, K = 105, T = 0.5, sigma = 30, r = 0.02))

6.549163351317259


### 10.1.5 CEV Model

In this coding example we show how the CEV model can be used to price a call option in Python:

In [ ]:
def cev_call(S_0, K, T, sigma, beta, r):
    v = 1 / (2 * (1 - beta))
    x_1 = 4 * (v**2) * (K ** (1 / v)) / ((sigma**2) * T)
    x_2 = 4 * (v**2) * ((S_0 * np.exp(r * T)) ** (1 / v)) / ((sigma**2) * T)
    kappa_1 = 2 * v + 2
    kappa_2 = 2 * v
    lambda_1 = x_2
    lambda_2 = x_1
    return np.exp(-r * T) * (
        (S_0 * np.exp(r * T) * (1 - ncx2.cdf(x_1, kappa_1, lambda_1)))
        - K * ncx2.cdf(x_2, kappa_2, lambda_2)
    )

print(cev_call(S_0=100, K=100, T=0.5, sigma=4, r=0.02, beta=0.5))

11.676110446149188


### 10.2.3 SABR Model

In the following coding example we detail how to calculate an approximate normal volatility using the asymptotic formula, given a set of SABR parameters:

In [ ]:
def sabr_normal_vol(S_0, K, T, sigma_0, alpha, beta, rho):

    c = lambda x: x**beta
    c_prime = lambda x: beta * (x ** (beta - 1))
    c_prime_prime = lambda x: beta * (beta - 1) * (x ** (beta - 2))
    S_mid = (S_0 + K) / 2
    gamma_1 = c_prime(S_mid) / c(S_mid)
    gamma_2 = c_prime_prime(S_mid) / c(S_mid)
    zeta = alpha * (S_0 ** (1 - beta) - K ** (1 - beta)) / (sigma_0 * (1 - beta))
    epsilon = T * (alpha**2)
    delta = np.log((np.sqrt(1 - 2 * rho * zeta + zeta**2) + zeta - rho) / (1 - rho))

    factor = alpha * (S_0 - K) / (delta)
    term_1 = ((2 * gamma_2 - gamma_1**2) / 24) * (sigma_0 * c(S_mid) / alpha) ** 2
    term_2 = rho * gamma_1 * sigma_0 * c(S_mid) / (4 * alpha)
    term_3 = (2 - 3 * (rho**2)) / 24
    return factor * (1 + epsilon * (term_1 + term_2 + term_3))


def sabr_call(S_0, K, T, sigma_0, r, alpha, beta, rho):
    assert S_0 != K
    vol = sabr_normal_vol(S_0, K, T, sigma_0, alpha, beta, rho)
    return bachelier_call(S_0, K, T, vol, r)


# Example usage
S_0 = 100
K = 105
T = 1
sigma_0 = 0.2
alpha = 0.3
beta = 0.5
rho = -0.3
r = 0.01

vol = sabr_normal_vol(S_0, K, T, sigma_0, alpha, beta, rho)
call_price = sabr_call(S_0, K, T, sigma_0, r, alpha, beta, rho)

print(f"SABR Normal Volatility: {vol}")
print(f"SABR Call Price: {call_price}")

SABR Normal Volatility: 1.994914617212177
SABR Call Price: 0.016653955102490608


### 10.4.1 Dupire's Formula

In this coding example, we show how the local volatility model developed by Dupire can be implemented in Python given a surface of available option prices and strikes:


In [ ]:
# synthetic data for strikes and expiries
strikes = np.linspace(80, 120, 10)      # 10 different strikes from 80 to 120
expiries = np.linspace(0.1, 2, 5)       # 5 different expiries from 0.1 to 2 years

# synthetic arbitrage-consistent call surface
S0 = 100.0
r = 0.0
sigma = 0.2
price_surface = np.array([
    [bs_call(S0, K, T, sigma, r) for K in strikes]
    for T in expiries
])

# expiry derivatives (10.53)
dT = expiries[2:] - expiries[:-2]
dC_dT = (price_surface[2:, :] - price_surface[:-2, :]) / dT.reshape(-1, 1)
expiry_derivatives = dC_dT[:, 1:-1]

# strike second derivatives (10.54)
dk = strikes[1] - strikes[0]
d2C_dK2 = (price_surface[:, 2:] - 2*price_surface[:, 1:-1] + price_surface[:, :-2]) / (dk**2)
strike_second_derivatives = d2C_dK2[1:-1, :]

# volatility surface (10.52)
K_interior = strikes[1:-1]
den = 0.5 * (K_interior**2) * strike_second_derivatives
variance_surface = expiry_derivatives / den
vol_surface = np.sqrt(variance_surface)


print("Price Surface:")
print(price_surface)
print("\nExpiry Derivatives:")
print(expiry_derivatives)
print("\nStrike Second Derivatives:")
print(strike_second_derivatives)
print("\nVariance Surface:")
print(variance_surface)
print("\nVolatility Surface:")
print(vol_surface)

Price Surface:
[[2.00002955e+01 1.55622733e+01 1.11837363e+01 7.09342065e+00
  3.76158254e+00 1.59206515e+00 5.23188860e-01 1.32373324e-01
  2.59209855e-02 3.98024808e-03]
 [2.04233411e+01 1.64823523e+01 1.28987822e+01 9.76929354e+00
  7.15379196e+00 5.06445937e+00 3.46848397e+00 2.30081997e+00
  1.48065081e+00 9.26063364e-01]
 [2.12814484e+01 1.77145243e+01 1.44967362e+01 1.16640128e+01
  9.23032905e+00 7.18841605e+00 5.51345837e+00 4.16836324e+00
  3.10928349e+00 2.29045687e+00]
 [2.21941828e+01 1.88697314e+01 1.58669013e+01 1.32001264e+01
  1.08700692e+01 8.86537261e+00 7.16530562e+00 5.74272257e+00
  4.56688749e+00 3.60588374e+00]
 [2.30826523e+01 1.99318962e+01 1.70775765e+01 1.45242819e+01
  1.22672941e+01 1.02942243e+01 8.58692590e+00 7.12340888e+00
  5.87956975e+00 4.83063538e+00]]

Expiry Derivatives:
[[2.26552737 3.4873683  4.81114968 5.75657527 5.89089569 5.25291528
  4.24841044 3.24564474]
 [2.51303066 3.12433599 3.61140299 3.9118708  4.0009613  3.89139122
  3.62305537 3.24